In [8]:
# import necessary libraries
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import keras

Using TensorFlow backend.


In [2]:
# note filepaths
PATH_TRAIN_LABELS = '../datasets/train_labels.csv'
PATH_TRAIN_RAW = '../datasets/train_images.npy'
PATH_TEST_RAW = '../datasets/test_images.npy'
PATH_TRAIN = '../datasets/train_images_cropped.npy'
PATH_TEST = '../datasets/test_images_cropped.npy'

In [3]:
# import datasets 
train_labels = np.array(pd.read_csv(PATH_TRAIN_LABELS, delimiter=",", header=0, index_col=0))
train_images = np.load(PATH_TRAIN, encoding="latin1")
test_images = np.load(PATH_TEST, encoding="latin1")

In [4]:
# note constants
IMG_SIZE = 100

In [5]:
# define labels
cat0 = sorted(['sink','pear','moustache','nose','skateboard','penguin','peanut','skull','panda',
        'paintbrush','nail','apple','rifle','mug','sailboat','pineapple','spoon','rabbit',
        'shovel','rollerskates','screwdriver','scorpion','rhinoceros','pool','octagon',
        'pillow','parrot','squiggle','mouth','empty','pencil'])
cat1 = {i: cat0[i] for i in range(len(cat0))}
lb = preprocessing.LabelBinarizer()
lb.fit(cat0)

# check labels
print(cat0)
print()
print(cat1)
print()
print(lb.classes_)

NameError: name 'preprocessing' is not defined

### DP

In [12]:
def new_data(data,method):
    ''' 
    function to go from dataset --> upto 3 preprocessed subimages 
    return the X given a method of choosing between the subimages
    (for example using find_largest_count or looking at ratios)
    we go from raw data to data that just has one subimage included
    input: raw image data
    return: chosen subimages data
    '''
    new_data = []
    for image in data:
        # get subimgs
        sub_images =  get_bounding_image(image.reshape(64,64))
        #find the largest one
        largest = method(sub_images)    
        #resize it
        dst = rescale(largest)
        #dst = resizeAndPad(largest, (28,28), padColor= 0) # previous way of rescaling img
        #save it
        new_data.append(dst)
    return np.asarray(new_data)

def find_largest_count_ratio(train_images):
    '''
    returns largest subimage based on the ratio of pixel counts between the actual subimage and the MNIST scaled version
    '''
    count_ratios=[]
    for img in images:
        pre_scaled_count = count_pixels_gray(img)
        post_scaled = rescale(img)
        post_scaled_count = count_pixels_gray(post_scaled)
        ratio = pre_scaled_count/post_scaled_count
        count_ratios.append( ratio )
    # find the max image based on ratio
    max_val = max(count_ratios)
    index = count_ratios.index(max_val) 
    return images[index]

new_x_train_data = new_data(train_images, find_largest_count_ratio)

NameError: name 'get_bounding_image' is not defined

In [10]:
"""
Simplistic implementation of the two-layer neural network.
Training method is stochastic (online) gradient descent with momentum.
As an example it computes XOR for given input.
Some details:
- tanh activation for hidden layer
- sigmoid activation for output layer
- cross-entropy loss
Less than 100 lines of active code.
"""

import time

n_hidden = 28
n_in = 784
n_out = 10
n_samples = 300

learning_rate = 0.01
momentum = 0.9

np.random.seed(0)

def flatten(data):
    flattened_data = []
    for entry in data:
    flattened_data.append(entry.flatten())
    return np.asarray(flattened_data)


def Sigmoid_forward(X):
     return 1.0 / (1.0 + np.exp(-X))

def Softmax_loss(X, y):
    num_examples = X.shape[0]
    probs = Softmax_predict(X)
    data_loss = 0
    for i in range(num_examples):
        corect_logprobs = -np.log(probs[i]*y[i])
        data_loss += corect_logprobs
    return 1./num_examples * data_loss
    #loss = -np.mean ( true * np.log(Y) + (1 - true) * np.log(1 - Y) )
    #return loss

def Softmax_predict(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def Softmax_diff(X, y):
    num_examples = X.shape[0]
    probs = Softmax_predict(X)
    probs[range(num_examples), y] -= 1
    return probs

def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def tanh_prime(x):
    return  1 - np.tanh(x)**2

def softmax_loss_naive(W, X, y, reg= 0):
    """
    Softmax loss function, naive implementation (with loops)
    Inputs:
    - W: C x D array of weights 
    - X: D x N array of data. Data are D-dimensional columns
    - y: 1-dimensional array of length N with labels 0...K-1, for K classes for us the one hot encoded
    - reg: (float) regularization strength
    Returns:
    a tuple of:
    - loss as single float
    - gradient with respect to weights W, an array of same size as W
    """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)

    #############################################################################
    # Compute the softmax loss and its gradient using explicit loops.           #
    # Store the loss in loss and the gradient in dW. If you are not careful     #
    # here, it is easy to run into numeric instability. Don't forget the        #
    # regularization!                                                           #
    #############################################################################

    # Get shapes
    num_classes = W.shape[0]
    num_train = X.shape[1]

    for i in range(num_train):
        # Compute vector of scores
        f_i = W.dot(X[:, i]) # in R^{num_classes}

        # Normalization trick to avoid numerical instability, per http://cs231n.github.io/linear-classify/#softmax
        log_c = np.max(f_i)
        f_i -= log_c

        # Compute loss (and add to it, divided later)
        # L_i = - f(x_i)_{y_i} + log \sum_j e^{f(x_i)_j}
        sum_i = 0.0
        for f_i_j in f_i:
          sum_i += np.exp(f_i_j)
        loss += -f_i[y[i]] + np.log(sum_i)

        # Compute gradient
        # dw_j = 1/num_train * \sum_i[x_i * (p(y_i = j)-Ind{y_i = j} )]
        # Here we are computing the contribution to the inner sum for a given i.
        for j in range(num_classes):
            p = np.exp(f_i[j])/sum_i
            dW[j, :] += (p-(j == y[i])) * X[:, i]

    # Compute average
    loss /= num_train
    dW /= num_train

    # Regularization
    loss += 0.5 * reg * np.sum(W * W)
    dW += reg*W

    return loss, dW


def train(x, true, V, W, bv, bw):

    # forward
    print (x.shape)
    print (np.dot(x, V).shape)

    A = np.dot(x, V) + bv
    Z = np.tanh(A)

    B = np.dot(Z, W) + bw
    Y = Softmax_predict(B)
    print (A.shape)
    print (Y.shape)

    # backward
    Ew, dW = softmax_loss_naive(W,x, true)
    print (Ew.shape)
    Ev = tanh_prime(A) * np.dot(W, Ew)

    #dW = Softmax_dW(x,true)
    dV = np.outer(x, Ev)

    loss = -np.mean ( true * np.log(Y) + (1 - true) * np.log(1 - Y) )

    # Note that we use error for each layer as a gradient
    # for biases

    return  loss, (dV, dW, Ev, Ew)

def predict(x, V, W, bv, bw):
    A = np.dot(x, V) + bv
    B = np.dot(np.tanh(A), W) + bw
    return (sigmoid(B) > 0.5).astype(int)


# Setup initial parameters
V = np.random.normal(scale=0.1, size=(n_in, n_hidden))
W = np.random.normal(scale=0.1, size=(n_hidden, n_out))
bv = np.zeros(n_hidden)
bw = np.zeros(n_out)
params = [V,W,bv,bw]

# Generate some data
X = flatten(new_x_train_data)
T = keras.utils.to_categorical(train_images, 10)

# Train
for epoch in range(100):
    err = []
    upd = [0]*len(params)
    t0 = time.clock()
    loss, grad = train(X, T, *params)
    params -= upd
    upd = learning_rate * grad + upd
    err.append( loss )

NameError: name 'new_x_train_data' is not defined